In [ ]:
import nltk
nltk.download('punkt')
nltk.download('brown')
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.corpus import brown
from nltk.corpus import indian
nltk.download('indian')
import collections
from bs4 import BeautifulSoup

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package indian to /root/nltk_data...
[nltk_data]   Package indian is already up-to-date!


In [ ]:
def read_test_data():
  sentences = list()
  golden_std = list()
  for i in range(1,13):
    file_name = "/content/Marathi_Dependency_Tree_Test_"+str(i)+".txt"
    soup = BeautifulSoup(open(file_name), "html.parser")
    text_tags = soup.find_all('sentence')


    for text in text_tags:
        word_list = list()
        std_list = list()
        sentence = text.get_text()
        sen_list = sentence.split("\n")
        error_flag = 0
        for word in sen_list:
          decomposed_parts = word.split("\t")
          if(len(decomposed_parts)==4 and decomposed_parts[1]!="(("):
            actual_tag = normalize_tag(decomposed_parts[2])
            if actual_tag == 'NULL' or actual_tag=='3':
              error_flag = 1

            word_list.append((decomposed_parts[1]))
            std_list.append((decomposed_parts[1],actual_tag))
        if error_flag != 1:
          golden_std.append(std_list)
        #print(word_list)
        #word_list = " ".join(word_list)
        #print(word_list)
          sentences.append(word_list)

  return sentences,golden_std

In [ ]:
def normalize_tag(tag):
  prev_char = '1'
  new_tag = str()
  for char in tag:
    if char == '_' and prev_char == '_':
      continue
    new_tag+=char
    prev_char = char
  return new_tag

In [ ]:
possible_tags = list()
observations = list()
tag_dict = dict()
starter_dict =  dict()
tx_matrix = list()
emission_matrix = dict()
tag_words = dict()
sentences = list()
for i in range(1,94):
  file_name = "/content/Marathi_Dependency_Tree_"+str(i)+".txt"
  soup = BeautifulSoup(open(file_name), "html.parser")
  text_tags = soup.find_all('sentence')


  for text in text_tags:
      word_list = list()
      sentence = text.get_text()
      sen_list = sentence.split("\n")
      error_flag = 0
      for word in sen_list:
        decomposed_parts = word.split("\t")
        if(len(decomposed_parts)==4 and decomposed_parts[1]!="(("):
          #print(decomposed_parts)

          actual_tag = normalize_tag(decomposed_parts[2])
          if actual_tag == 'NULL' or actual_tag == '3':
            error_flag = 1
          word_list.append((decomposed_parts[1],actual_tag))
      if error_flag != 1:
        sentences.append(word_list)


print(sentences)
print(len(sentences))


[[('काही', 'QT_QTF'), ('कंपन्यांनी', 'N_NN'), ('वस्तूंचे', 'N_NN'), ('वजन', 'N_NN'), ('कमी', 'QT_QTF'), ('व', 'CC_CCD'), ('किंमत', 'N_NN'), ('पूर्वीप्रमाणेच', 'RB'), ('ठेवून', 'V_VM'), ('माल', 'N_NN'), ('बाजारात', 'N_NN'), ('आणण्याचा', 'V_VM'), ('प्रयत्न', 'N_NN'), ('सुरू', 'JJ'), ('केला', 'V_VM'), ('आहे', 'V_VAUX'), ('.', 'RD_PUNC')], [('त्यामुळे', 'CC_CCD'), ('ग्राहकांना', 'N_NN'), ('जादा', 'QT_QTF'), ('पैसे', 'N_NN'), ('मोजावे', 'V_VM'), ('लागत', 'V_VM'), ('आहेत', 'V_VAUX'), ('.', 'RD_PUNC')], [('बिलात', 'N_NN'), ('वाढीव', 'JJ'), ('वजन', 'N_NN'), ('लिहिल्यास', 'V_VM'), ('कारवाई', 'N_NN'), ('होऊ', 'V_VM'), ('शकते', 'V_VAUX'), (',', 'RD_PUNC'), ('असे', 'PR_PRP'), ('वैधमापनशास्त्र', 'N_NNP'), ('विभागाने', 'N_NN'), ('म्हटले', 'V_VM'), ('आहे', 'V_VAUX'), ('.', 'RD_PUNC')], [('सध्या', 'N_NST'), ('बाजारात', 'N_NN'), ('प्रचंड', 'JJ'), ('स्पर्धा', 'N_NN'), ('आहे', 'V_VM'), ('.', 'RD_PUNC')], [('उत्पादन', 'N_NN'), ('खर्च', 'N_NN'), ('व', 'CC_CCD'), ('उत्पन्न', 'N_NN'), ('यांचा', 'PR_PRP'), ('

In [ ]:
possible_tags = list()
#calculate all possible tags
for sent in sentences:
  for word in sent:
    if word[1] not in possible_tags:
      possible_tags.append(word[1])
print(possible_tags)


for tag in possible_tags:
  tag_dict[tag] = 0
  tag_words[tag] = list()
  starter_dict[tag] = 0
for sentence in sentences:
  for word in sentence:
    tag_dict[word[1]] += 1
    tag_words[word[1]].append(word[0])
for tag,lists in tag_words.items():
  tag_words[tag] = dict(collections.Counter(lists))

#emission probability - only include non zero values
for tag,words in tag_words.items():
  for word,count in words.items():
    new_count = count/tag_dict[tag]
    words[word] = new_count

print(tag_dict)
print(tag_words)
#print("Answer:",tag_words['QT_QTF'])

#starting probability -
denominator = len(sentences)
for sentence in sentences:
  starter_dict[sentence[0][1]]+=1
for tag,counts in starter_dict.items():
  starter_dict[tag] = (counts/denominator)
print(starter_dict)

#transition probability -
pair_counts = dict()
for tag in possible_tags:
  pair_counts[tag] = dict()
  for other_tag in possible_tags:
    pair_counts[tag][other_tag] = 0
#print(pair_counts)
for sentence in sentences:
  for i in range(1,len(sentence)):
    pair_counts[sentence[i-1][1]][sentence[i][1]] += 1
print(pair_counts)
for current_tag,next_tag_dict in pair_counts.items():
  #print(type(next_tag_dict),next_tag_dict)
  for tag,count in next_tag_dict.items():
    new_count = count/tag_dict[current_tag]
    next_tag_dict[tag] = new_count
print(pair_counts)
#pair counts = transition, tag_words = emission

['QT_QTF', 'N_NN', 'CC_CCD', 'RB', 'V_VM', 'JJ', 'V_VAUX', 'RD_PUNC', 'PR_PRP', 'N_NNP', 'N_NST', 'QT_QTC', 'CC_CCS', 'DM_DMR', 'DM_DMD', 'RP_RPD', 'V_VM_VNG', 'RD_SYM', 'PR_PRQ', 'PR_PRL', 'RP_NEG', 'QT_QTO', 'PR_PRF', 'V_VM_VF', 'V_VM_VNF', 'V_VM_VINF', 'RP_INTF', 'CC_CCS_UT', 'DM_DMQ', 'XC', 'PSP', 'RP_INJ', 'PR_PRC', 'PP', 'UT', 'RD_UNK', 'RD_RDF', 'VM', 'NN', 'VAUX', 'RBP', 'INTF', 'RP_CL', 'RD_ECH']
{'QT_QTF': 1351, 'N_NN': 25740, 'CC_CCD': 2253, 'RB': 825, 'V_VM': 13213, 'JJ': 5459, 'V_VAUX': 4322, 'RD_PUNC': 11183, 'PR_PRP': 4319, 'N_NNP': 10808, 'N_NST': 1494, 'QT_QTC': 2469, 'CC_CCS': 514, 'DM_DMR': 470, 'DM_DMD': 3007, 'RP_RPD': 96, 'V_VM_VNG': 28, 'RD_SYM': 1258, 'PR_PRQ': 371, 'PR_PRL': 31, 'RP_NEG': 244, 'QT_QTO': 229, 'PR_PRF': 132, 'V_VM_VF': 49, 'V_VM_VNF': 45, 'V_VM_VINF': 7, 'RP_INTF': 150, 'CC_CCS_UT': 230, 'DM_DMQ': 70, 'XC': 13, 'PSP': 105, 'RP_INJ': 17, 'PR_PRC': 4, 'PP': 3, 'UT': 18, 'RD_UNK': 26, 'RD_RDF': 28, 'VM': 11, 'NN': 2, 'VAUX': 1, 'RBP': 1, 'INTF': 1, 

In [ ]:
#viterbi matrix
def testing_output(word_list,possible_tags,tag_words,starter_dict):
  word_values_dict = dict()
  for i in range(len(word_list)):
    word_values_dict[i] = dict()
    for tag in possible_tags:
      word_values_dict[i][tag] = (0,'.')  #replace by laplace smoothing - should not be zero

  #for all words in the sentence
  for i in range(len(word_list)):
    current_word = word_list[i]
    #print(current_word)
    exists_in_vocab = False
    for tag in possible_tags:
      if current_word not in tag_words[tag].keys():
        continue
      exists_in_vocab = True

      emission_prob = tag_words[tag][current_word]
      transition_prob = starter_dict[tag]
      if i != 0:
        max_val = 0
        max_state = '.'
        for prev_tag in possible_tags:
          curr_val = word_values_dict[i-1][prev_tag][0]*pair_counts[prev_tag][tag] #worried
          #print(word_values_dict[i-1][prev_tag][0],pair_counts[prev_tag][tag])
          if curr_val>max_val:
            max_val = curr_val
            max_state = prev_tag
          #print(current_word,tag,prev_tag,curr_val)
        word_values_dict[i][tag] = (emission_prob*max_val,max_state)
        #print(current_word,tag,emission_prob,word_values_dict[i][tag],max_val)


      else:
        word_values_dict[i][tag] = (emission_prob*transition_prob,'q0')
    #if word not in vocab, then assume it's a noun
    if exists_in_vocab == False:
      max_val = 0
      max_state = '.'
      #print(current_word)
      if i != 0:
        for prev_tag in possible_tags:
            curr_val = word_values_dict[i-1][prev_tag][0]*pair_counts[prev_tag]['N_NN'] #worried
            if curr_val>max_val:
              max_val = curr_val
              max_state = prev_tag
        word_values_dict[i]['N_NN'] = (0.01,max_state)
      else:
        word_values_dict[i]['N_NN'] = (starter_dict['N_NN'],'q0')
      #print(max_state)
    #print(word_values_dict[i])
  answer = list()
  last_dist = word_values_dict[len(word_list)-1]
  #print(last_dist)
  #print(answer)
  max_tag = '.'
  max_val = 0
  for tag in possible_tags:
    if last_dist[tag][0]>max_val:
      max_val = last_dist[tag][0]
      max_tag = tag
  answer.append(max_tag)
  #print(word_list)
  try:
    prev_tag = last_dist[max_tag][1]
  except:
    print("ERROR")
    return 1,[]
  answer.append(prev_tag)
  for i in range(len(word_list)-2,-1,-1):
    #print(i,answer)
    try:
      next_prev = word_values_dict[i][prev_tag][1]
    except:
      print("ERROR")
      return 1,[]
    answer.append(next_prev)
    prev_tag = next_prev
  answer.reverse()
  final_ans = list()
  i = 1
  for word in word_list:
    final_ans.append((word,answer[i]))
    i+=1
  #print(final_ans)
  return 0,final_ans

In [ ]:
def evaluate(ans,gold):
  #print(ans,gold)
  correct = 0
  total = len(ans)
  for i in range(len(ans)):
    if ans[i][1] == gold[i][1]:
      correct+=1
  return correct/total

In [ ]:
test,golden = read_test_data()
print("Testing:",test)
total = 0
unk = 0
sum_ratio = 0
for sentence in test:

  word_list = sentence
  #print(word_list)
  result,ans = testing_output(word_list,possible_tags,tag_words,starter_dict)
  if result == 1:
    unk+=1
  else:
    ratio = evaluate(ans,golden[total])
    #if ratio<=0.6:
      #print("ANSWER:",ans)
      #print("GOLDEN:",golden[total])
    if ratio == -1:
      unk+=1
    else:
      sum_ratio+=ratio
  total+=1
print("Total:",total)
print("Unk:",unk)
print("Accuracy:",sum_ratio/(total-unk))


Testing: [['त्या', 'वेळी', 'दंगलविरोधी', 'पथकाला', '(', 'वज्र', ')', 'पोलिसांनी', 'पाचारण', 'केले', '.'], ['अनेक', 'वरिष्ठ', 'पोलिस', 'अधिकाऱ्यांनी', 'घटनास्थळाला', 'भेट', 'दिली', '.'], ['दगडफेकीच्या', 'या', 'घटनेमुळे', 'स्टेशन', 'परिसरातील', 'सर्व', 'दुकाने', 'बंद', 'झाली', 'होती', '.'], ['या', 'घटनेबाबत', 'श्री', '.', 'शिंदे', 'म्हणाले', ',', '"', 'गेल्या', 'पंधरा', 'दिवसांपासून', 'भोसले', 'यांच्याकडून', 'आमच्या', 'कार्यकर्त्यांना', 'धमक्\u200dया', 'येत', 'आहेत', '.'], ['त्यांच्या', 'कार्यकर्त्यांनी', 'आज', 'माझी', 'भावजय', 'साधना', 'प्रवीण', 'शिंदे', 'यांनाही', 'धक्काबुक्की', 'केली', '.'], ['याबाबत', 'पोलिसांनी', 'तक्रार', 'दिल्यावरही', 'दखल', 'घेतली', 'नाही'], ["'", "'", 'दगडफेकीचा', 'प्रसाद', 'श्री', '.', 'शिंदे', 'यांनाही', 'बसल्याचे', 'त्यांनी', 'पत्रकारांना', 'दाखविले', '.'], ['या', 'घटनेबाबत', 'चौघांना', 'ताब्यात', 'घेण्यात', 'आल्याचे', 'बंडगार्डन', 'पोलिसांनी', 'सांगितले', '.'], ['बाणेर', '-', 'बालेवाडी', 'प्रभागात', 'मारामारी', 'बाणेर', 'बालेवाडी', 'प्रभागातील', '"', 'बाणेर'